In [56]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_palette('husl')

In [57]:
df = pd.read_pickle('data/processed.pickle')
df.head()

,key,title,authors,abstract,year_published,tags,category,skills,skill_cat,types,lang_family
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,2021,"[approach:fully_automated, data_available:Fals...",[dynamic],[correctness],[correctness_dynamic],"[description, evaluation]",[OOP]
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,2021,"[approach:semi_automatic, interaction:unknown,...","[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]","[description, evaluation]",[OOP]
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,2021,"[skill:readability, approach:fully_automated, ...",[static],"[readability, maintainability]","[maintainability_static, readability_static]",[description],[OOP]
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,2021,"[approach:semi_automatic, data_available:False...","[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]","[description, evaluation]",[Unknown]
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,2021,"[approach:fully_automated, data_available:Fals...","[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]","[description, evaluation]",[OOP]


# Discussion Stats
## Grading
### Correctness

In [58]:
correct_df = df.copy(deep=True)
correct_df.skills = correct_df.skills.astype(str)
correct_df = correct_df[correct_df.skills.str.contains("^\['correctness'\]$")]
correct_df['tech'] = correct_df.tags.apply(lambda labels: [label for label in labels if label.startswith('technique')])
correct_df[['key', 'title', 'skill_cat', 'tech']]

,key,title,skill_cat,tech
0,rayyan-354359269,An experience of automated assessment in a lar...,[correctness_dynamic],"[technique:unit_testing, technique:pre_defined..."
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"[correctness_dynamic, correctness_static]","[technique:model_solution_req, technique:patte..."
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"[correctness_dynamic, correctness_static]","[technique:output_matching, technique:pattern_..."
4,rayyan-354359273,Analysis of an automatic grading system within...,"[correctness_dynamic, correctness_static]","[technique:pattern_matching, technique:unit_te..."
5,rayyan-354359274,Evaluating Control-Flow Graph Similarity for G...,[correctness_static],"[technique:model_solution_req, technique:patte..."
...,...,...,...,...
114,rayyan-354359408,ArTEMiS - An automatic assessment management s...,"[correctness_dynamic, correctness_static]","[technique:ci_cd, technique:unit_testing, tech..."
115,rayyan-354359409,Linkage objects for generalized instruction in...,"[correctness_ml, correctness_static]","[technique:machine_learning, technique:pattern..."
116,rayyan-354359410,Automated data-driven hints for computer progr...,"[correctness_dynamic, correctness_static]","[technique:unit_testing, technique:static_anal..."
117,rayyan-354359411,Learning and teaching numerical methods with a...,"[correctness_dynamic, correctness_static]","[technique:matching_rules, technique:unit_test..."


In [59]:
(correct_df.shape[0] / df.shape[0]) * 100

66.11570247933885

In [60]:
correct_df.skill_cat.value_counts()

[correctness_dynamic]                        36
[correctness_dynamic, correctness_static]    21
[correctness_static]                         17
[correctness_ml, correctness_static]          3
[correctness_ml]                              2
[correctness_dynamic, correctness_ml]         1
Name: skill_cat, dtype: int64

In [61]:
(correct_df.category.value_counts() / correct_df.shape[0]) * 100

[dynamic]            45.00
[dynamic, static]    26.25
[static]             21.25
[static, ml]          3.75
[ml]                  2.50
[dynamic, ml]         1.25
Name: category, dtype: float64

In [62]:
correct_df['only_dynamic'] = df.category.apply(lambda x: 'dynamic' in str(x) and 'static' not in str(x) and 'ml' not in str(x))
(correct_df[correct_df.only_dynamic].explode('tech').tech.value_counts() / correct_df[correct_df.only_dynamic].shape[0]) * 100

technique:unit_testing                100.000000
technique:ci_cd                        11.111111
technique:model_solution_req           11.111111
technique:trace_examination             5.555556
technique:property_based_testing        5.555556
technique:output_matching               5.555556
technique:pre_defined_questions         2.777778
technique:unit_test_expansion           2.777778
technique:mutation_testing              2.777778
technique:novel                         2.777778
technique:test_case_generation          2.777778
technique:gui_testing                   2.777778
technique:program_synthesis             2.777778
technique:code_repair_for_feedback      2.777778
Name: tech, dtype: float64

In [63]:
correct_df['only_static'] = df.category.apply(lambda x: 'dynamic' not in str(x) and 'static' in str(x) and 'ml' not in str(x))
(correct_df[correct_df.only_static].explode('tech').tech.value_counts() / correct_df[correct_df.only_static].shape[0]) * 100

technique:pattern_matching            70.588235
technique:model_solution_req          41.176471
technique:model_solution_closeness    35.294118
technique:program_repair              35.294118
technique:code_repair_for_feedback    29.411765
technique:clustering                  17.647059
technique:matching_rules              17.647059
technique:trace_examination           17.647059
technique:output_matching             11.764706
technique:graph_analysis              11.764706
technique:control_flow_graph           5.882353
technique:data_driven                  5.882353
technique:token_pattern_matching       5.882353
technique:knowledge_base               5.882353
technique:dynamic_analysis             5.882353
Name: tech, dtype: float64

In [64]:
correct_df['dynamic_static'] = df.category.apply(lambda x: 'dynamic' in str(x) and 'static' in str(x) and 'ml' not in str(x))
(correct_df[correct_df.dynamic_static].explode('tech').tech.value_counts() / correct_df[correct_df.dynamic_static].shape[0]) * 100

technique:unit_testing                100.000000
technique:matching_rules               33.333333
technique:pattern_matching             28.571429
technique:static_analysis              23.809524
technique:model_solution_req           19.047619
technique:style_check                  19.047619
technique:program_repair               14.285714
technique:model_solution_closeness      9.523810
technique:dynamic_analysis              9.523810
technique:output_matching               9.523810
technique:ci_cd                         9.523810
technique:fault_localisation            4.761905
technique:program_verification          4.761905
technique:code_repair_for_feedback      4.761905
technique:language_stuctures            4.761905
technique:test_case_generation          4.761905
technique:knowledge_base                4.761905
Name: tech, dtype: float64

In [65]:
correct_df['ml'] = df.category.apply(lambda x: 'ml' in str(x))
ml_df = correct_df.loc[correct_df.ml]
ml_df[['key', 'title', 'category']]

,key,title,category
13,rayyan-354359285,An Intelligent Online Judge System for Program...,"[dynamic, ml]"
19,rayyan-354359295,Convolutional neural network applied to code a...,[ml]
34,rayyan-354359310,Automatic assessment of programming assignment...,[ml]
61,rayyan-354359341,Source-Code Similarity Measurement: Syntax Tre...,"[static, ml]"
83,rayyan-354359367,Targeted Example Generation for Compilation Er...,"[static, ml]"
115,rayyan-354359409,Linkage objects for generalized instruction in...,"[static, ml]"


### Maintainability

In [66]:
maint_df = df.copy(deep=True)
maint_df.skills = maint_df.skills.astype(str)
maint_df = maint_df[maint_df.skills.str.contains("^\['maintainability'\]$")].copy()
maint_df

,key,title,authors,abstract,year_published,tags,category,skills,skill_cat,types,lang_family
92,rayyan-354359378,A Comparison of Inquiry-Based Conceptual Feedb...,"Cordova L., Carver J., Gershmel N., Walia G.",The feedback provided by current testing educa...,2021,"[skill:test_suite_quality, approach:fully_auto...",[static],['maintainability'],[maintainability_static],[evaluation],[OOP]


In [67]:
maint_df['tech'] = maint_df.tags.apply(lambda labels: [label for label in labels if label.startswith('technique')])
(maint_df.explode('tech').tech.value_counts() / maint_df.shape[0]) * 100

technique:model_solution_req    100.0
technique:static_analysis       100.0
Name: tech, dtype: float64

### Readability

In [68]:
# Analysing hint based problem solving strategy among novice programmers through gamification technique - Might be removed
read_df = df.copy(deep=True)
read_df.skills = read_df.skills.astype(str)
read_df = read_df[read_df.skills.str.contains("^\['readability'\]$")]
read_df[['key', 'title', 'category']]

,key,title,category
30,rayyan-354359306,The impact of automated code quality feedback ...,[static]
77,rayyan-354359360,Promoting Code Quality via Automated Feedback ...,[static]
79,rayyan-354359362,Measure Students’ Contribution in Web Programm...,[static]
107,rayyan-354359398,Static analyses in python programming courses,[static]


In [69]:
(read_df.shape[0] / df.shape[0]) * 100

3.3057851239669422

In [70]:
read_df.category.value_counts()

[static]    4
Name: category, dtype: int64

### Correctness, Documentation

In [71]:
doc_df = df.copy(deep=True)
doc_df.skills = doc_df.skills.astype(str)
doc_df[doc_df.skills.str.contains("^\['correctness', 'documentation'\]$")]

,key,title,authors,abstract,year_published,tags,category,skills,skill_cat,types,lang_family
29,rayyan-354359305,Developing applications to automatically grade...,Gerdes J.,There are many unique challenges associated wi...,2017,"[approach:fully_automated, data_available:Fals...",[static],"['correctness', 'documentation']","[correctness_static, documentation_static]","[description, evaluation]",[OOP]


### Combination Graders

In [ ]:
(df.skills.value_counts() / df.shape[0]) * 100

In [ ]:
(df.skill_cat.value_counts() / df.shape[0]) * 100

In [ ]:
combo_df = df.copy(deep=True)
combo_df.skills = combo_df.skills.astype(str)
combo_df[combo_df.skills.str.contains("^\['correctness', 'readability', 'maintainability']$")]

### Code Repair papers

In [ ]:
cr_df = df.copy()
cr_df['tech'] = cr_df.tags.apply(lambda labels: [label for label in labels if 'code_repair' in label or 'program_repair' in label])
cr_df.tech = cr_df.tech.map(lambda labels: None if len(labels) == 0 else labels)
cr_df[~cr_df.tech.isna()]

## Degree of Automation

In [ ]:
doa_df = df.copy()
doa_df['approach'] = doa_df.tags.apply(lambda labels: [str(label) for label in labels if label.startswith('approach')])
doa_df = doa_df.explode('approach')
doa_df.approach.value_counts()

In [ ]:
(doa_df.approach.value_counts() /  doa_df.shape[0]) * 100

In [ ]:
(doa_df[doa_df.approach.str.contains('fully')].skills.value_counts() / doa_df.shape[0]) * 100

#### CI/CD

In [ ]:
df.explode('tags')[df.explode('tags').tags.str.contains('ci_cd')]

#### Semi

In [ ]:
doa_df[doa_df.approach.str.contains('semi')].explode('tags').tags.value_counts()

## Language Paradigms

In [ ]:
lp_df = df.copy()
lp_df.lang_family.value_counts()

In [ ]:
(lp_df.lang_family.value_counts() / lp_df.shape[0]) * 100

In [ ]:
lp_df.lang_family = lp_df.lang_family.astype(str)
lp_df[lp_df.lang_family.str.contains('Query')]

## Evaluation

In [ ]:
eval_df = df.copy()
eval_df.tags = eval_df.tags.apply(lambda labels: [str(label) for label in labels if label.startswith('evaluation')])
eval_df = eval_df.explode('tags')
(eval_df.tags.value_counts() / eval_df.shape[0]) * 100

In [ ]:
eval_df[eval_df.tags.str.contains('manual_grading')]

In [100]:
df.explode('tags')[df.explode('tags').tags.str.contains('ci_cd')]

,key,title,authors,abstract,year_published,tags,category,skills,skill_cat,types,lang_family
17,rayyan-354359293,Improving Programming Education Quality with A...,"Cai Y.-Z., Tsai M.-H.","As the rapid growth of information technology,...",2019,technique:ci_cd,[dynamic],[correctness],[correctness_dynamic],"[description, evaluation]",[OOP]
36,rayyan-354359313,Enhancing the Learning of Database Access Prog...,B. Pérez,Database access programming is a noteworthy co...,2021,technique:ci_cd,[dynamic],[correctness],[correctness_dynamic],"[description, evaluation]",[OOP]
67,rayyan-354359348,Effectiveness of Real-Time Feedback and Instru...,Lee HH,"In this paper, we present our findings on enha...",2021,technique:ci_cd,[dynamic],[correctness],[correctness_dynamic],"[description, evaluation]",[OOP]
79,rayyan-354359362,Measure Students’ Contribution in Web Programm...,B. -A. Nguyen and K. -Y. Ho and H. -M. Chen,Large scale software systems are usually devel...,2020,technique:ci_cd,[static],[readability],[readability_static],"[description, evaluation]",[Web]
80,rayyan-354359363,Investigating the Essential of Meaningful Auto...,Q. Hao and J. P. Wilson and C. Ottaway and N. ...,This study investigated the essential of meani...,2019,technique:ci_cd,"[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]",[evaluation],[OOP]
96,rayyan-354359385,Towards understanding the effective design of ...,"Hao Q., Smith IV D.H., Ding L., Ko A., Ottaway...",Background and Context: automated feedback for...,2021,technique:ci_cd,[dynamic],[correctness],[correctness_dynamic],[evaluation],[OOP]
114,rayyan-354359408,ArTEMiS - An automatic assessment management s...,"Krusche S., Seitz A.",The increasing number of students in computer ...,2018,technique:ci_cd,"[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]","[description, evaluation]",[OOP]


#### Semi

In [ ]:
doa_df[doa_df.approach.str.contains('semi')].explode('tags').tags.value_counts()

## Language Paradigms

In [ ]:
lp_df = df.copy()
lp_df.lang_family.value_counts()

In [ ]:
(lp_df.lang_family.value_counts() / lp_df.shape[0]) * 100

In [ ]:
lp_df.lang_family = lp_df.lang_family.astype(str)
lp_df[lp_df.lang_family.str.contains('Query')]

## Evaluation

In [ ]:
eval_df = df.copy()
eval_df.tags = eval_df.tags.apply(lambda labels: [str(label) for label in labels if label.startswith('evaluation')])
eval_df = eval_df.explode('tags')
(eval_df.tags.value_counts() / eval_df.shape[0]) * 100

In [ ]:
eval_df[eval_df.tags.str.contains('manual_grading')]